Finetune and deploy a custom Generative Command model

* Note: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
* Ensure that IAM role used has AmazonSageMakerFullAccess
* To deploy this ML model successfully, ensure that:
Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used:
aws-marketplace:ViewSubscriptions
aws-marketplace:Unsubscribe
aws-marketplace:Subscribe
or your AWS account has a subscription to the packages for Cohere Command Finetuning. If so, skip step: Subscribe to the finetune algorithm

1. Subscribe to the finetune algorithm
To subscribe to the model algorithm:

Open the algorithm listing page Cohere Command Finetuning
On the AWS Marketplace listing, click on the Continue to Subscribe button.
On the Subscribe to this software page, review and click on "Accept Offer" if you and your organization agrees with EULA, pricing, and support terms.
Once you click on Continue to configuration button and then choose a region, you will see a Product Arn displayed. This is the algorithm ARN that you need to specify while creating a finetune or deploying the finetuned model as an endpoint using boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
%pip install -U cohere-aws

from cohere_aws import Client
import boto3
import sagemaker as sage
from sagemaker.s3 import S3Uploader

In [ ]:
region = boto3.Session().region_name

cohere_package = "cohere-command-ft-v2-1-03b5a2b33e113acdbde5f8a142f85db7"

# Mapping for algorithms
algorithm_map = {
    "us-east-1": f"arn:aws:sagemaker:us-east-1:865070037744:algorithm/{cohere_package}",
    "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:algorithm/{cohere_package}",
    "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:algorithm/{cohere_package}",
    "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:algorithm/{cohere_package}",
    "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:algorithm/{cohere_package}",
    "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:algorithm/{cohere_package}",
    "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:algorithm/{cohere_package}",
    "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:algorithm/{cohere_package}",
}
if region not in algorithm_map.keys():
    raise Exception(f"Current boto3 session region {region} is not supported.")

arn = algorithm_map[region]

In [ ]:
s3_data_dir = "s3://input//"
sess = sage.Session()
train_dataset = S3Uploader.upload("../Data/finetuning_data.jsonl", s3_data_dir, sagemaker_session=sess)

Specify a directory on S3 where finetuned models should be stored. Make sure you do not reuse the same directory across multiple runs.

In [ ]:
# TODO update this with a custom S3 path
# DO NOT re-use the same s3 directory for multiple finetunes
# DO NOT add a trailing slash at the end

s3_models_dir = "s3://..." 

In [ ]:
#Create Cohere Client
co = Client(region_name=region)

In [ ]:
train_parameters = {
    "train_epochs": 10,
    "strategy": "vanilla",
    "train_batch_size": 100,
    "early_stopping_patience": 5,
    "early_stopping_threshold": 0.001,
}

In [ ]:
# This will take approximately 30 minutes with the example dataset
finetune_name = "sample-finetune"
co.create_finetune(arn=arn,
    name=finetune_name,
    train_data=train_dataset,
    s3_models_dir=s3_models_dir,
    instance_type="ml.p4de.24xlarge",
    training_parameters=train_parameters,
)

The finetuned weights for the above will be store in a tar file {s3_models_dir}/sample-finetune.tar.gz where the file name is the same as the name used during the creation of the finetune.

In [ ]:
#Create a Endpoint
co.create_endpoint(arn=arn,
    endpoint_name="command-finetune-test",
    s3_models_dir=s3_models_dir,
    recreate=True,
    instance_type="ml.p4d.24xlarge",
)

# If the endpoint is already created, you just need to connect to it
# co.connect_to_endpoint(endpoint_name="command-finetune-test")

In [ ]:
#Perform real time inference
prompt = "Classify the following text as either very negative, negative, neutral, positive or very positive: mr. deeds is , as comedy goes , very silly -- and in the best way."

result = co.generate(prompt=prompt, max_tokens=50)


In [ ]:
#delete the endpoint
co.delete_endpoint()
co.close()